In [10]:
from collections import namedtuple
from types import SimpleNamespace
import attr
from io import BytesIO
import json
image_loc: str = '/z-aparatu/20240721_134802.heic'

import imageio
import ollama
import base64

# display the image
from IPython.display import display, Image as DisplayImage

import imageio.v3 as iio
from PIL import Image as PILImage

def heic_to_base64(heic_file_path):
    image = imageio.imread(heic_file_path)

    pil_image = PILImage.fromarray(image)

    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")

    img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_base64


def display_heic_image(image_path) -> None:
    # Read the HEIC image
    heic_image = iio.imread(image_path)

    # Convert the numpy array to a PIL image
    pil_image = PILImage.fromarray(heic_image)

    # Save the PIL image to a BytesIO object in JPEG format
    img_buffer = BytesIO()
    pil_image.save(img_buffer, format='JPEG')
    img_buffer.seek(0)  # Rewind the buffer to the beginning

    # Display the image
    display(DisplayImage(data=img_buffer.read(), format='jpeg'))

queries = [
    'Describe this scene in very detailed way',
    'extract and interpret text from this image',
    'identify and classify objects in this image'
]


def dict_to_obj(dictionary):
    result = json.loads(json.dumps(dictionary), object_hook=lambda d: SimpleNamespace(**d))
    return result

def run_chat(msg: str) -> str:
  res = ollama.chat(
      stream=False,
    model="llava:v1.6",
    messages=[
      {
        'role': 'system',
        'content': 'You are helping to describe tags related to provided pictures. Avoid redundant words like image'
      },
      {
        'role': 'user',
        'content': 'msg',
        'images': [heic_to_base64(image_loc)]
      }
    ],
    options = {
      #'temperature': 1.5, # very creative
      'temperature': 0 # very conservative (good for coding and correct syntax)
    }
  )

  response = dict_to_obj(res)
  return response.message.content

# display_heic_image(image_loc)

for query in queries:
    display(query)
    display(run_chat(query))



'Describe this scene in very detailed way'

/tmp/ipykernel_1297/837728070.py:19: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(heic_file_path)


' The image shows a person sitting at an outdoor dining table. There are various items on the table, including what appears to be a bowl of food and some saucers with condiments or possibly small plates. The setting suggests a casual dining experience, possibly in a restaurant or café patio area. The person is wearing glasses and has short hair. They seem to be enjoying their meal or perhaps waiting for someone else to join them. '

'extract and interpret text from this image'

' The image shows a person sitting at an outdoor dining table. There are various items on the table, including what appears to be a bowl of food and some saucers with condiments or possibly small plates. The setting suggests a casual dining experience, possibly in a restaurant or café patio area. The person is wearing glasses and has short hair. They seem to be enjoying their meal or perhaps waiting for someone else to join them. '

'identify and classify objects in this image'

' The image shows a person sitting at an outdoor dining table. There are various items on the table, including what appears to be a bowl of food and some saucers with condiments or possibly small plates. The setting suggests a casual dining experience, possibly in a restaurant or café patio area. The person is wearing glasses and has short hair. They seem to be enjoying their meal or perhaps waiting for someone else to join them. '